# Introduction to programming and numerical analysis - Summer examination 2023

This exam is made by:
- Jacob Grenaa Vestergaard
- Emil Henry Erbas
- Jeppe Vanderhaegen

# Importing essentials

In [1]:
# Importing autoreload
%load_ext autoreload
%autoreload 2

# Importing essential packages
from types import SimpleNamespace
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize_scalar
import tabulate
from scipy.optimize import minimize

# Importing classes
from examproject import ClassQ1
from examproject import GriewankOptimizer

# Defining the models in this Notebook
modelQ1 = ClassQ1()

# Problem 1

**Question 1.1**

In the class "ClassQ1", we have included the code required to answer the question. The function maximize_utility() is utilized to maximize the utility and determine the optimal labor hours. By varying the values of G, we solve the utility maximization problem. Finally, we utilize np.isclose to compare our obtained solution with the analytical solution provided in the exam assignment.

Using np.isclose allows us to verify whether the provided analytical solution effectively solves the maximization problem by ensuring that the values are in agreement.

In [2]:
# Values for G
G_values = [1.0,2.0]

for G in G_values:
    modelQ1 = ClassQ1()

    # Assigning values
    modelQ1.par.G = G
    
    # Calling the provided analytical soltion
    L_optimal = modelQ1.par.optimal_L

    # Solving the model
    maximized_L = modelQ1.maximize_utility()
    # Printing the value
    print("When G is equal to",G,", optimal L is", f'{maximized_L[0]:.8f}')    
    
    if np.isclose(maximized_L[0], L_optimal,rtol=1e-3):
            print("In this scenario, the values obtained from the minmizer-solution and analytical-solution are approximately the same.")
    else:
            print("The values are not close.")
    print()

When G is equal to 1.0 , optimal L is 15.29938165
In this scenario, the values obtained from the minmizer-solution and analytical-solution are approximately the same.

When G is equal to 2.0 , optimal L is 15.29937604
In this scenario, the values obtained from the minmizer-solution and analytical-solution are approximately the same.



**Question 1.2**

We generate various real wage values using np.linspace. Then, for each real wage value, we utilize the provided analytical solution to calculate the optimal labor supply, using the optimal_labor_supply() function.

Next, we store the results in an empty list. Finally, we plot the real wage on the x-axis and the corresponding optimal labor supply on the y-axis.

The graph displays an anticipated outcome. Initially, with a low real wage, the gradient is steep, and subsequently, the optimal labor supply gradually converges towards a stable level, when the real wage increases.

In [ ]:
# Define the range of real wage values
omega_values = np.linspace(0.1, 2.0, 100)

# Calculate the corresponding optimal labor supply choices for each wage value
L_star_values = []
for omega in omega_values:

    modelQ1 = ClassQ1()
    modelQ1.par.omega_tilde = (1 - modelQ1.par.tau) * omega
    L_star = modelQ1.optimal_labor_supply()
    L_star_values.append(L_star)

# Plot the relationship between w and L_star
plt.plot(omega_values, L_star_values)
plt.xlabel('Real Wage')
plt.ylabel('Optimal Labor Supply')
plt.title('Optimal Labor Supply vs. Real Wage')
plt.grid(True)
plt.show()   

**Question 1.3**

The method used is similar to that in Question 1.2. We generate different tau values using np.linspace. For each tau value, we calculate the optimal labor supply, government consumption, and worker utility within a loop.

Next, we create empty lists to store all the results.

We proceed to create three plots, with tau plotted on the x-axis.

The outcomes are as follows: labor supply decreases significantly for high tax levels, government consumption exhibits a maximum point, and worker utility decreases for larger tau values.

In [ ]:
# Define the range of tau values
tau_values = np.linspace(0.1, 0.9, 100)

# Initialize arrays to store the calculated values
L_values = []
G_values = []
utility_values = []

# Calculate the corresponding L, G, and worker utility for each tau value
for tau in tau_values:
    modelQ1 = ClassQ1()
    modelQ1.par.omega_tilde = (1 - tau) * modelQ1.par.omega
    L_star = modelQ1.optimal_labor_supply()
    G = tau * modelQ1.par.omega * L_star * modelQ1.par.omega_tilde
    utility = np.log((modelQ1.par.kappa + (1 - tau) * modelQ1.par.omega * L_star)**modelQ1.par.alpha * G**(1 - modelQ1.par.alpha)) - modelQ1.par.nu * L_star**2
    L_values.append(L_star)
    G_values.append(G)
    utility_values.append(utility)

# Create subplots with 1 row and 3 columns
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Plot 1: Labor Supply vs. Tax Rate
axs[0].plot(tau_values, L_values)
axs[0].set_xlabel('Tau (Tax Rate)')
axs[0].set_ylabel('Labor Supply (L)')
axs[0].set_title('Labor Supply vs. Tax Rate')
axs[0].grid(True)

# Plot 2: Government Consumption vs. Tax Rate
axs[1].plot(tau_values, G_values)
axs[1].set_xlabel('Tau (Tax Rate)')
axs[1].set_ylabel('Government Consumption (G)')
axs[1].set_title('Government Consumption vs. Tax Rate')
axs[1].grid(True)

# Plot 3: Worker Utility vs. Tax Rate
axs[2].plot(tau_values, utility_values)
axs[2].set_xlabel('Tau (Tax Rate)')
axs[2].set_ylabel('Worker Utility')
axs[2].set_title('Worker Utility vs. Tax Rate')
axs[2].grid(True)

# Adjust spacing between subplots
plt.tight_layout()

# Display the subplots
plt.show()


**Question 1.4**

As one could observe in Question 1.3, there is a specific value of tau between 0.2 and 0.4 that maximizes worker utility.

To facilitate this calculation, we introduce a new function called calculate_utility() that computes the utility obtained for various tau values.

Subsequently, we utilize the minimize function from scipy to determine the tau value that maximizes worker utility.

The result of our analysis reveals that tau = 0.34 corresponds to the optimal value that maximizes worker utility.

In [ ]:
# Function to calculate worker utility for a given tau value
def calculate_utility(tau):
    modelQ1 = ClassQ1()
    modelQ1.par.omega_tilde = (1 - tau) * modelQ1.par.omega
    L_star = modelQ1.optimal_labor_supply()
    G = tau * modelQ1.par.omega * L_star * modelQ1.par.omega_tilde
    utility = np.log((modelQ1.par.kappa + (1 - tau) * modelQ1.par.omega * L_star) ** modelQ1.par.alpha * G ** (1 - modelQ1.par.alpha)) - modelQ1.par.nu * L_star ** 2
    return -utility  # We minimize the negative utility to maximize the original utility

# Find the optimal tau using scipy's minimize_scalar function
result = minimize_scalar(calculate_utility, bounds=(0.01, 1), method='bounded')
optimal_tau = result.x
max_utility = -result.fun

print("The socially optimal tax rate maximizing worker utility is tau =", optimal_tau)

# Plot the relationship between tau and worker utility
utility_values = [-calculate_utility(tau) for tau in tau_values]  # Calculate utility for all tau values
plt.plot(tau_values, utility_values)
plt.axvline(x=optimal_tau, color='r', linestyle='--', label='Optimal Tau')
plt.xlabel('Tau (Tax Rate)')
plt.ylabel('Worker Utility')
plt.title('Worker Utility vs. Tax Rate')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

**Question 1.5**

Explain the code

In [ ]:
# Baseline parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * (16 ** 2))
omega = 1.0
tau = 0.3432216374902929  # Tau found in Q1.3
government = 3.4
epsilon = 1

# Set1 parameters
sigma_set1 = 1.001
rho_set1 = 1.001

# Utility function for set1 parameters
def objective_function_q5_set1(L):
    C = kappa + (1 - tau) * omega * L
    value = (((alpha * C ** ((sigma_set1 - 1) / sigma_set1) + (1 - alpha) * government ** (
                (sigma_set1 - 1) / sigma_set1)) ** (sigma_set1 / (sigma_set1 - 1))) ** (1 - rho_set1) - 1) / (
                     1 - rho_set1) - nu * (L ** (1 + epsilon)) / (1 + epsilon)
    return -value  # negative because we want to maximize

# Finding optimal labor hours for set1 parameters
def maximize_utility_q5_set1():
    result_q5_set1 = minimize_scalar(objective_function_q5_set1, bounds=(0, 100), method='bounded')
    return result_q5_set1.x

# Setting up government constraint
def constraint(government, L_optimal):
    return government - tau * omega * L_optimal * (1 - tau) * omega

# Function to minimize
def objective_function(government):
    return abs(constraint(government, L_optimal))

# Bisection method to find the government value
def find_government_value():
    a = 0.0  # Lower bound of government value
    b = 10.0  # Upper bound of government value

    # Find the root using bounded method
    result = minimize_scalar(objective_function, bounds=(a, b), method='bounded')

    if result.success:
        return result.x
    else:
        return None

# Run the code
L_optimal = maximize_utility_q5_set1()
government_value = find_government_value()

if government_value is not None:
    G_residual = constraint(government_value, L_optimal)
    print("With the parameters from set1, we obtain the following results:")
    print("Optimal labor hours:", L_optimal)
    print("Government value:", government_value)
    print("G_residual:", G_residual)
else:
    print("Failed to find a government value that satisfies the constraint.")

In [ ]:
# Baseline parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * (16 ** 2))
omega = 1.0
tau = 0.3432216374902929  # Tau found in Q1.3
government = 3.4
epsilon = 1

# Set2 parameters
sigma_set2 = 1.5
rho_set2 = 1.5

# Utility function for set2 parameters
def objective_function_q5_set2(L):
    C = kappa + (1 - tau) * omega * L
    value = (((alpha * C ** ((sigma_set2 - 1) / sigma_set2) + (1 - alpha) * government ** (
                (sigma_set2 - 1) / sigma_set2)) ** (sigma_set2 / (sigma_set2 - 1))) ** (1 - rho_set2) - 1) / (
                     1 - rho_set2) - nu * (L ** (1 + epsilon)) / (1 + epsilon)
    return -value  # negative because we want to maximize

# Finding optimal labor hours for set2 parameters
def maximize_utility_q5_set2():
    result_q5_set2 = minimize_scalar(objective_function_q5_set2, bounds=(0, 100), method='bounded')
    return result_q5_set2.x

# Setting up government constraint
def constraint(government, L_optimal):
    return government - tau * omega * L_optimal * (1 - tau) * omega

# Function to minimize
def objective_function(government):
    return abs(constraint(government, L_optimal))

# Bisection method to find the government value
def find_government_value():
    a = 0.0  # Lower bound of government value
    b = 10.0  # Upper bound of government value

    # Find the root using bounded method
    result = minimize_scalar(objective_function, bounds=(a, b), method='bounded')

    if result.success:
        return result.x
    else:
        return None

# Run the code
L_optimal = maximize_utility_q5_set2()
government_value = find_government_value()

if government_value is not None:
    G_residual = constraint(government_value, L_optimal)
    print("With the parameters from set2, we obtain the following results:")
    print("Optimal labor hours:", L_optimal)
    print("Government value:", government_value)
    print("G_residual:", G_residual)
else:
    print("Failed to find a government value that satisfies the constraint.")

**Question 1.6**

Explain the code

In [ ]:
# Utility function for set1 parameters
def objective_function_q5_set1(L, tau):
    C = kappa + (1 - tau) * omega * L
    value = (((alpha * C ** ((sigma_set1 - 1) / sigma_set1) + (1 - alpha) * government ** (
                (sigma_set1 - 1) / sigma_set1)) ** (sigma_set1 / (sigma_set1 - 1))) ** (1 - rho_set1) - 1) / (
                     1 - rho_set1) - nu * (L ** (1 + epsilon)) / (1 + epsilon)
    return -value  # negative because we want to maximize

# Function to minimize
def objective_function(tau):
    L_optimal = maximize_utility_q5_set1(tau)
    G_residual = constraint(government, L_optimal)
    return abs(G_residual)

# Finding optimal labor hours for set1 parameters and given tau
def maximize_utility_q5_set1(tau):
    result_q5_set1 = minimize_scalar(objective_function_q5_set1, bounds=(0, 100), args=(tau,), method='bounded')
    return result_q5_set1.x

# Bisection method to find the tau value
def find_tau_value():
    a = 0.0  # Lower bound of tau value
    b = 1.0  # Upper bound of tau value

    # Find the root using bounded method
    result = minimize_scalar(objective_function, bounds=(a, b), method='bounded')

    if result.success:
        return result.x
    else:
        return None

# Run the code
tau_value = find_tau_value()

if tau_value is not None:
    L_optimal = maximize_utility_q5_set1(tau_value)
    government_value = constraint(government, L_optimal)
    print("With the parameters from set1, we obtain the following results:")
    print("Optimal labor hours:", L_optimal)
    print("Government value:", government_value)
    print("G_residual:", G_residual)
    print("Socially optimal tax rate:", tau_value)

In [ ]:
# Utility function for set2 parameters
def objective_function_q5_set2(L, tau):
    C = kappa + (1 - tau) * omega * L
    value = (((alpha * C ** ((sigma_set2 - 1) / sigma_set2) + (1 - alpha) * government ** (
                (sigma_set2 - 1) / sigma_set2)) ** (sigma_set2 / (sigma_set2 - 1))) ** (1 - rho_set2) - 1) / (
                     1 - rho_set2) - nu * (L ** (1 + epsilon)) / (1 + epsilon)
    return -value  # negative because we want to maximize

# Function to minimize
def objective_function(tau):
    L_optimal = maximize_utility_q5_set2(tau)
    G_residual = constraint(government, L_optimal)
    return abs(G_residual)

# Finding optimal labor hours for set2 parameters and given tau
def maximize_utility_q5_set2(tau):
    result_q5_set2 = minimize_scalar(objective_function_q5_set2, bounds=(0, 100), args=(tau,), method='bounded')
    return result_q5_set2.x

# Bisection method to find the tau value
def find_tau_value():
    a = 0.0  # Lower bound of tau value
    b = 1.0  # Upper bound of tau value

    # Find the root using bounded method
    result = minimize_scalar(objective_function, bounds=(a, b), method='bounded')

    if result.success:
        return result.x
    else:
        return None

# Run the code
tau_value = find_tau_value()

if tau_value is not None:
    L_optimal = maximize_utility_q5_set2(tau_value)
    government_value = constraint(government, L_optimal)
    print("With the parameters from set2, we obtain the following results:")
    print("Optimal labor hours:", L_optimal)
    print("Government value:", government_value)
    print("G_residual:", G_residual)
    print("Socially optimal tax rate:", tau_value)

# Problem 2

**Question 2.1**

Explain the code


In [ ]:
code

**Question 2.2**

Explain the code


In [ ]:
code

**Question 2.3**

Explain the code

In [ ]:
code

**Question 2.4**

Explain the code

In [ ]:
code

# Problem 3

**Question 2.5**

Explain the code

**Question 3.1**

We implement the Griewank optimization algorithm in the class "GriewankOptimizer". The class takes inputs such as bounds, tolerance, warm-up iterations, and maximum iterations as inputs. The class includes functions for calculating the Griewank function, refining the global optimization, and plotting the results.

The refined_global_optimizer() function performs the optimization by iteratively generating random initial points and updating them based on the Griewank function's values. After a warm-up period, it uses the minimize function from the SciPy library to refine the initial points. The best solution found so far is stored and compared with each new solution obtained. The process continues until the maximum number of iterations is reached or the tolerance level is met.

Finally, the run_optimization() function executes the optimization process, prints the best solution found, and displays a plot showing the progression of the initial guesses (x0) over the iterations.

In [ ]:
# Create an instance of the GriewankOptimizer class
bounds = [-600, 600]
tol = 1e-8
warmup_iter = 10
max_iter = 1000
optimizer = GriewankOptimizer(bounds, tol, warmup_iter, max_iter)

# Run the optimization
optimizer.run_optimization()

**Question 3.2**

We utilize the refined global optimizer with multi-start in our analysis once again. It is executed for different values of $\underline{K}$, specifically 10 and 100, to compare the number of iterations required to find the global minimum. We also examine different seeds due to the uniform distribution of vector x, as randomness could potentially impact the outcome.

Upon comparing the results between $\underline{K} = 10$ and $\underline{K} = 100$, it is evident that, in general, $\underline{K} = 10$ outperforms the latter. This is indicated by the requirement of fewer iterations to identify the global minimum.

When $\underline{K} = 100$, the increased number of warm-up iterations allows the optimizer to explore the solution space more extensively before applying the refinement strategy. However, overall, this approach proves to be inefficient.

In [ ]:
def griewank(x):
    return griewank_(x[0], x[1])

def griewank_(x1, x2):
    A = x1**2 / 4000 + x2**2 / 4000
    B = np.cos(x1 / np.sqrt(1)) * np.cos(x2 / np.sqrt(2))
    return A - B + 1

def refined_global_optimizer(bounds, tol, warmup_iter, max_iter, seed):
    np.random.seed(seed)
    x_best = None

    for k in range(max_iter):
        x = np.random.uniform(bounds[0], bounds[1], size=2)

        if k >= warmup_iter:
            chi = 0.5 * (2 / (1 + np.exp((k - warmup_iter) / 100)))
            x0 = chi * x + (1 - chi) * x_best
            res = minimize(griewank, x0, method='BFGS', tol=tol)
        else:
            res = minimize(griewank, x, method='BFGS', tol=tol)

        x_best = res.x if k == 0 or res.fun < griewank(x_best) else x_best

        if griewank(x_best) < tol:
            break

    return x_best, k+1

# Settings
bounds = [-600, 600]
tol = 1e-8
max_iter = 1000
K_values = [10,100]
seed_values = [2000, 2001, 2002, 2003, 2004]  # Different seed values to iterate over

# Table headers
headers = ["Seed", "K", "Convergence Iterations", "x_best", "f(x_best)"]

# Table rows
table = []

for seed in seed_values:
    for K in K_values:
        # Run optimizer with current K and seed
        warmup_iter = K
        result, iterations = refined_global_optimizer(bounds, tol, warmup_iter, max_iter, seed)
        f_x_best = griewank(result)

        # Append row to the table
        table.append([seed, K, iterations, result.round(4), f_x_best])

# Print the table
print(tabulate.tabulate(table, headers=headers, tablefmt="grid"))
